Test modelu na zbiorze CIFAR

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm  # Importowanie tqdm do paska postępu

# Sprawdzenie czy mamy dostęp do GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Przygotowanie transformacji dla CIFAR-10 (MobileNet wymaga obrazów 224x224)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Pobieranie zbioru testowego CIFAR-10
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Pobieranie pretrenowanego modelu MobileNetV2
model = torchvision.models.mobilenet_v2(pretrained=True)

# Model pretrenowany na ImageNet klasyfikuje 1000 klas, ale my go nie zmieniamy, chcemy zobaczyć wynik bez zmiany
model = model.to(device)

# Funkcja testująca model
def test_model():
    model.eval()  # Ustawienie modelu w tryb ewaluacji
    correct = 0
    total = 0
    
    # Dodanie paska postępu do pętli testowej
    with torch.no_grad():
        for data in tqdm(test_loader, desc="Testing MobileNetV2"):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)  # Forward pass
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Test Accuracy (bez trenowania na CIFAR-10): {100 * correct / total}%')

# Wywołanie funkcji testowej
test_model()


Douczenie modelu na zbiorze CIFAR

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt

# Sprawdzenie dostępności GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Przygotowanie transformacji dla CIFAR-10
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Pobieranie zbioru danych CIFAR-10
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Pobieranie modelu MobileNetV2 pretrenowanego na ImageNet
model = torchvision.models.mobilenet_v2(pretrained=True)

# Dodanie Dropout po warstwie global_avg_pool (możesz eksperymentować z poziomem dropout)
class MobileNetV2WithDropout(nn.Module):
    def __init__(self, model):
        super(MobileNetV2WithDropout, self).__init__()
        self.features = model.features
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(p=0.3)  # Dodany Dropout
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.3),  # Dropout przed klasyfikacją
            nn.Linear(model.last_channel, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.classifier(x)
        return x

# Przystosowanie modelu do klasyfikacji 10 klas CIFAR-10
model = MobileNetV2WithDropout(model).to(device)

# Funkcje strat i optymalizator
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Przygotowanie zmiennych do przechowywania strat i dokładności
train_losses = []
test_accuracies = []

# Funkcja testująca model
def test_model():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy}%')
    return accuracy

# Trening modelu z progress barem i zapisaniem statystyk
def train_model(epochs=1):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader))
        for i, data in progress_bar:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # Zerowanie gradientów
            optimizer.zero_grad()

            # Forward + backward + optymalizacja
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Aktualizacja progress bara
            progress_bar.set_description(f'Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / (i + 1):.4f}')
        
        # Zapis strat po każdej epoce
        train_losses.append(running_loss / len(train_loader))
        accuracy = test_model()
        test_accuracies.append(accuracy)

# Uruchomienie treningu na 5 epokach
train_model(epochs=5)

# Rysowanie wykresów strat i dokładności
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(test_accuracies, label='Test Accuracy')
plt.title('Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.show()
